In [1]:
import pandas as pd
import os
import glob

# Specify the path to subfolders with CSV files
main_path = r'C:\ResearchFiles\AllFiles_Cleaned'

# List of buildings to include
selected_buildings = ['EBU3B']

# Recursively search for all CSV files in the subfolders of the selected buildings
all_files = []
for building in selected_buildings:
    building_path = os.path.join(main_path, building)
    all_files.extend(glob.glob(os.path.join(building_path, "*.csv")))

# Combine all files into a single DataFrame
df_list = []
for filename in all_files:
    df = pd.read_csv(filename)
    df['smart_plug'] = os.path.basename(filename).split('_')[1]  # Add a column for the smart plug identifier
    df['building'] = os.path.basename(os.path.dirname(filename))  # Add a column for the building
    df_list.append(df)

# Combine all the dataframes into one
combined_df = pd.concat(df_list, ignore_index=True)

In [2]:
# Drop unneccesary columns
combined_df = combined_df.drop(columns = ['analogInput_2', 'analogInput_4', 'analogInput_5', 'binaryInput_3', 'binaryValue_1', 'building'])

In [3]:
# Convert timestamp column to datetime
combined_df['time'] = pd.to_datetime(combined_df['time'])

In [4]:
# Create a new column for the 15 minute time interval
combined_df['15min'] = combined_df['time'].dt.floor('15T')

In [5]:
combined_df

,time,analogInput_3,smart_plug,15min
0,2023-09-14 18:16:00,1904.0,180408,2023-09-14 18:15:00
1,2023-09-14 18:17:00,1560.0,180408,2023-09-14 18:15:00
2,2023-09-14 18:18:00,1785.0,180408,2023-09-14 18:15:00
3,2023-09-14 18:19:00,2057.0,180408,2023-09-14 18:15:00
4,2023-09-14 18:20:00,2057.0,180408,2023-09-14 18:15:00
...,...,...,...,...
25738141,2024-06-10 23:55:00,0.0,944300,2024-06-10 23:45:00
25738142,2024-06-10 23:56:00,0.0,944300,2024-06-10 23:45:00
25738143,2024-06-10 23:57:00,0.0,944300,2024-06-10 23:45:00
25738144,2024-06-10 23:58:00,0.0,944300,2024-06-10 23:45:00


In [6]:
# Group by 'smart_plug' and '15min', then sum the power readings
min15_df = combined_df.groupby(['smart_plug', '15min']).agg({'analogInput_3': 'sum'}).reset_index()

# Rename 'power_reading' to 'average_power_usage'
min15_df.rename(columns={'analogInput_3': 'average_power_usage'}, inplace=True)

# The 'min15_df' now contains the 15 min interval total power usage for each plug in the selected buildings

In [7]:
min15_df

,smart_plug,15min,average_power_usage
0,180408,2023-09-14 18:15:00,26075.0
1,180408,2023-09-14 18:30:00,39545.0
2,180408,2023-09-14 18:45:00,192324.0
3,180408,2023-09-14 19:00:00,28517.0
4,180408,2023-09-14 19:15:00,28002.0
...,...,...,...
1726493,944300,2024-06-10 22:45:00,0.0
1726494,944300,2024-06-10 23:00:00,0.0
1726495,944300,2024-06-10 23:15:00,0.0
1726496,944300,2024-06-10 23:30:00,0.0


In [8]:
# Read in the helper data with building name, load type, and manual inspection status
plug_info = pd.read_csv(r'C:\Users\vaugh\Downloads\helper_spreadsheet(2).csv')

In [9]:
plug_info

,smart_plug,building_name,Load_Type,Inspection
0,291824,Atkinson Hall,Computer,disconnected
1,291956,Atkinson Hall,Printer,checked
2,292032,Atkinson Hall,Printer,checked
3,183436,Atkinson Hall,Water Dispenser,disconnected
4,181084,Atkinson Hall,TV,checked
...,...,...,...,...
112,283884,EBU3B,Printer,checked
113,288480,EBU3B,Printer,checked
114,291792,EBU3B,Printer,missing
115,565612,EBU3B,Printer,checked


In [10]:
# Convert the plug id columns for both dataframes to strings to prepare for merging
min15_df['smart_plug'] = min15_df['smart_plug'].astype(str)
plug_info['smart_plug'] = plug_info['smart_plug'].astype(str)

# Merge the aggregated data with the plug info data
merged_15min = pd.merge(min15_df, plug_info, on='smart_plug')
merged_15min = merged_15min[
    (merged_15min['Inspection'] == 'checked') & 
    (~merged_15min['smart_plug'].isin(['299184', '183436', '186204', '634584', '284068', '298808', 
                                       '180552', '944300', '186212', '291792', '283892', '288848', 
                                       '565996', '283884', '287196', '654200', '286460', '182136', 
                                       '297624', '181084', '291824', '284416', '293688', '291080', 
                                       '290240', '297984', '289840', '182552', '291728', '180672', 
                                       '183428', '284556', '285120', '291076', '291740', '183232', 
                                       '284372', '180940', '182268', '182584', '292032', '294576', 
                                       '296488', '298776', '299852', '639140', '651712', '652512', 
                                       '301192', '562240', '566540', '780512']))
]

# Rename the time column more accurately
merged_15min = merged_15min.rename(columns = {'15min': 'Timestamp'})

In [11]:
merged_15min

,smart_plug,Timestamp,average_power_usage,building_name,Load_Type,Inspection
0,180408,2023-09-14 18:15:00,26075.0,EBU3B,Printer,checked
1,180408,2023-09-14 18:30:00,39545.0,EBU3B,Printer,checked
2,180408,2023-09-14 18:45:00,192324.0,EBU3B,Printer,checked
3,180408,2023-09-14 19:00:00,28517.0,EBU3B,Printer,checked
4,180408,2023-09-14 19:15:00,28002.0,EBU3B,Printer,checked
...,...,...,...,...,...,...
1705203,782264,2024-06-10 22:45:00,1224031.0,EBU3B,Printer,checked
1705204,782264,2024-06-10 23:00:00,120531.0,EBU3B,Printer,checked
1705205,782264,2024-06-10 23:15:00,121818.0,EBU3B,Printer,checked
1705206,782264,2024-06-10 23:30:00,917158.0,EBU3B,Printer,checked


In [12]:
# Create new df with a column for the total building load from all plugs at each timestamp
total_building_load = merged_15min.groupby(['building_name', 'Timestamp']).agg({'average_power_usage': 'sum'}).reset_index()

In [13]:
total_building_load

,building_name,Timestamp,average_power_usage
0,EBU3B,2023-09-14 18:15:00,9.310813e+06
1,EBU3B,2023-09-14 18:30:00,5.978398e+06
2,EBU3B,2023-09-14 18:45:00,1.097936e+07
3,EBU3B,2023-09-14 19:00:00,1.042595e+07
4,EBU3B,2023-09-14 19:15:00,1.518046e+07
...,...,...,...
22527,EBU3B,2024-06-10 22:45:00,1.085142e+07
22528,EBU3B,2024-06-10 23:00:00,1.425518e+07
22529,EBU3B,2024-06-10 23:15:00,9.822876e+06
22530,EBU3B,2024-06-10 23:30:00,1.360368e+07


In [14]:
# Read in the data for the EBU3B submetering
ebu3b_submetering = pd.read_csv(r'C:\Users\vaugh\Downloads\helper_spreadsheet(4).csv')

# Remove unneccessary columns so that we're left with just the timestamp, elevator submetering load, total lights submetering load, and total servers submetering load
ebu3b_submetering = ebu3b_submetering.drop(columns = ['Lights 1st Floor', 'Lights 2nd Floor', 'Lights 3rd Floor', 'Lights 4th Floor', 'WARREN.EBU3B_1st_Floor_E2545#Real Power Mean#kW', 'WARREN.EBU3B_1st_Floor_E2546#Real Power Mean#kW', 'WARREN.EBU3B_1st_Floor_E2548#Real Power Mean#kW', 'WARREN.EBU3B_E2544#Real Power Mean#kW', 'WARREN.EBU3B_Panel_M_E2526#Real Power Mean#kW'])

# Convert timestamp column to datetime
ebu3b_submetering['Timestamp'] = pd.to_datetime(ebu3b_submetering['Timestamp'])

In [15]:
ebu3b_submetering

,Timestamp,Elevator,Total Lights,Total Servers
0,2023-01-01 00:15:00,0.629064,7.893459,102.036439
1,2023-01-01 00:30:00,0.658419,7.896329,102.235192
2,2023-01-01 00:45:00,0.504297,7.924650,102.482906
3,2023-01-01 01:00:00,5.773901,8.140687,102.294906
4,2023-01-01 01:15:00,5.650762,7.918884,101.985907
...,...,...,...,...
54521,2024-07-21 23:30:00,0.838737,11.850697,94.435776
54522,2024-07-21 23:45:00,4.860091,11.810819,94.522451
54523,2024-07-22 00:00:00,6.950442,11.724997,94.519934
54524,NaT,NaN,0.000000,0.000000


In [16]:
# Merge the submetering and plug load dfs on the Timestamp column
both_merged_15min = pd.merge(total_building_load, ebu3b_submetering, on='Timestamp')

# Convert the plug load data's milliWatt values to kilowatts for continuity with the metering data
both_merged_15min['average_power_usage'] = both_merged_15min['average_power_usage'] / 1_000_000

# Convert all power usage values from KW to KW/hour
both_merged_15min['average_power_usage'] = both_merged_15min['average_power_usage'] / 4
both_merged_15min['Elevator'] = both_merged_15min['Elevator'] / 4
both_merged_15min['Total Lights'] = both_merged_15min['Total Lights'] / 4
both_merged_15min['Total Servers'] = both_merged_15min['Total Servers'] / 4

In [17]:
both_merged_15min

,building_name,Timestamp,average_power_usage,Elevator,Total Lights,Total Servers
0,EBU3B,2023-09-14 18:15:00,2.327703,0.490538,5.495005,20.976975
1,EBU3B,2023-09-14 18:30:00,1.494600,1.456425,5.385257,20.997443
2,EBU3B,2023-09-14 18:45:00,2.744839,0.502262,5.234220,20.911499
3,EBU3B,2023-09-14 19:00:00,2.606487,0.928332,5.366518,20.835240
4,EBU3B,2023-09-14 19:15:00,3.795115,0.501778,5.572761,20.860299
...,...,...,...,...,...,...
22527,EBU3B,2024-06-10 22:45:00,2.712855,0.617818,4.274012,22.588832
22528,EBU3B,2024-06-10 23:00:00,3.563796,0.754441,4.419412,22.563593
22529,EBU3B,2024-06-10 23:15:00,2.455719,1.283662,4.288537,22.624248
22530,EBU3B,2024-06-10 23:30:00,3.400919,1.206065,4.151662,22.599077


In [18]:
# Read in the building metering data with the real total energy consumptuon from the entirety of EBU3B
building_metering = pd.read_csv(r'C:\Users\vaugh\Downloads\building_metering.csv')

# Drop unneccessary columns so that we're just left with the timestamp and total energy consumption from EBU3B
building_metering = building_metering.drop(columns = ['WARREN.CAL_IT_E2531#Real Power Mean#kW', 'WARREN.CAL_IT_E2532#Real Power Mean#kW', 'WARREN.CAL_IT_Gate_E2530#Real Power Mean#kW', 'WARREN.EBU3B_E2520#Real Power Mean#kW', 'WARREN.EBU3B_E2521#Real Power Mean#kW', 'Atkinson total'])

# Convert timestamp column to datetime
building_metering['Timestamp'] = pd.to_datetime(building_metering['Timestamp'])

In [19]:
building_metering

,Timestamp,EBU3B total
0,2023-01-01 00:15:00,312.613174
1,2023-01-01 00:30:00,313.511780
2,2023-01-01 00:45:00,317.042526
3,2023-01-01 01:00:00,318.071701
4,2023-01-01 01:15:00,316.035248
...,...,...
54521,2024-07-21 23:30:00,397.910767
54522,2024-07-21 23:45:00,396.301117
54523,2024-07-22 00:00:00,394.709076
54524,NaT,0.000000


In [29]:
# Merge all three datasets together to include plugload usage, submetering data, and total building metering data
all_merged_15min = pd.merge(both_merged_15min, building_metering, on='Timestamp')

# Rename the plugload usage column for better readability
all_merged_15min.rename(columns = {'average_power_usage': 'average_plug_load'}, inplace=True)

# Dropping building name column and setting Timestamp as index for better readability
all_merged_15min = all_merged_15min.drop(columns = ['building_name'])
all_merged_15min = all_merged_15min.set_index('Timestamp')

In [30]:
all_merged_15min

,average_plug_load,Elevator,Total Lights,Total Servers,EBU3B total
Timestamp,,,,,
2023-09-14 18:15:00,2.327703,0.490538,5.495005,20.976975,311.203552
2023-09-14 18:30:00,1.494600,1.456425,5.385257,20.997443,303.462402
2023-09-14 18:45:00,2.744839,0.502262,5.234220,20.911499,303.112823
2023-09-14 19:00:00,2.606487,0.928332,5.366518,20.835240,307.819702
2023-09-14 19:15:00,3.795115,0.501778,5.572761,20.860299,304.548492
...,...,...,...,...,...
2024-06-10 22:45:00,2.712855,0.617818,4.274012,22.588832,372.360016
2024-06-10 23:00:00,3.563796,0.754441,4.419412,22.563593,381.564133
2024-06-10 23:15:00,2.455719,1.283662,4.288537,22.624248,385.190674


In [22]:
# Calculate the Pearson correlation coefficient for lights submetering vs. average plug load
lights_plugs_correlation = all_merged_15min['Total Lights'].corr(all_merged_15min['average_plug_load'])

print(f"The Pearson correlation coefficient between the average plug load in EBU3B and the lights submetering is: {lights_plugs_correlation}")

The Pearson correlation coefficient between the average plug load in EBU3B and the lights submetering is: 0.09226563085375943


In [23]:
# Calculate the Pearson correlation coefficient for elevator submetering vs. average plug load
elevator_plugs_correlation = all_merged_15min['Elevator'].corr(all_merged_15min['average_plug_load'])

print(f"The Pearson correlation coefficient between the average plug load in EBU3B and the elevator submetering is: {elevator_plugs_correlation}")

The Pearson correlation coefficient between the average plug load in EBU3B and the elevator submetering is: -0.04008337272749189


In [24]:
# Calculate the Pearson correlation coefficient for elevator submetering vs. lights submetering
elevator_lights_correlation = all_merged_15min['Elevator'].corr(all_merged_15min['Total Lights'])

print(f"The Pearson correlation coefficient between the lights submetering in EBU3B and the elevator submetering is: {elevator_lights_correlation}")

The Pearson correlation coefficient between the lights submetering in EBU3B and the elevator submetering is: 0.6190652712882163


In [25]:
# Calculate the Pearson correlation coefficient for elevator submetering vs. total metering load
elevator_total_correlation = all_merged_15min['Elevator'].corr(all_merged_15min['EBU3B total'])

print(f"The Pearson correlation coefficient between the elevator submetering and the total power consumption in EBU3B is: {elevator_total_correlation}")

The Pearson correlation coefficient between the elevator submetering and the total power consumption in EBU3B is: 0.3009439064501389


In [26]:
# Calculate the Pearson correlation coefficient for lights submetering vs. total metering load
lights_total_correlation = all_merged_15min['Total Lights'].corr(all_merged_15min['EBU3B total'])

print(f"The Pearson correlation coefficient between the lights submetering and the total power consumption in EBU3B is: {lights_total_correlation}")

The Pearson correlation coefficient between the lights submetering and the total power consumption in EBU3B is: 0.06017390607190594


In [27]:
# Calculate the Pearson correlation coefficient for servers submetering vs. total metering load
servers_total_correlation = all_merged_15min['Total Servers'].corr(all_merged_15min['EBU3B total'])

print(f"The Pearson correlation coefficient between the servers submetering and the total power consumption in EBU3B is: {servers_total_correlation}")

The Pearson correlation coefficient between the servers submetering and the total power consumption in EBU3B is: 0.2466114495551074


In [31]:
# Saving the finalized dataframe as a csv file to my GitHub Repository for future reference
all_merged_15min.to_csv(r'C:\Users\vaugh\Desktop\smart-plug-research\All-Combined-Datasets.csv')